In [17]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
import kaleido


In [18]:
DATA = Path(r"C:\Users\Rubah\OneDrive\سطح المكتب\week 2\data\processed\analytics_table.parquet")
FIGS = Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

In [19]:
def save_fig(fig, path: Path, *, scale: int = 2):
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)


In [20]:
df = pd.read_parquet(DATA)


In [21]:
print("Rows:", len(df))
print("Cols:", len(df.columns))
print(df.dtypes.head(15))
print(df.isna().sum().sort_values(ascending=False).head(10))

Rows: 5
Cols: 15
order_id                   string[python]
user_id                    string[python]
amount                            Float64
quantity                            Int64
created_at            datetime64[ns, UTC]
status                             object
date                               object
year                              float64
month                      string[python]
dow                                object
hour                              float64
country                            object
signup_date                        object
amount_winsor                     Float64
amount__is_outlier                boolean
dtype: object
amount                1
created_at            1
quantity              1
date                  1
dow                   1
month                 1
year                  1
amount__is_outlier    1
amount_winsor         1
hour                  1
dtype: int64


# Q1
Revenue by country

In [22]:
rev = (
    df.groupby("country", dropna=False)
    .agg(
        n=("order_id","size"),
        revenue=("amount","sum"),
        aov=("amount","mean")
    )
    .reset_index()
    .sort_values("revenue", ascending=False)
)
print(rev)

  country  n  revenue     aov
1      SA  4    145.5  36.375
0      AE  1      0.0    <NA>


In [23]:
fig = px.bar(rev, x="country", y="revenue", title="Revenue by country (all data)")
fig.update_layout(title={"x":0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")

save_fig(fig, FIGS / "revenue_by_country.png")
fig.show()

# Q2
Revenue trend by month

In [25]:
if "month" not in df.columns and "created_at" in df.columns:
    dt = pd.to_datetime(df["created_at"], errors="coerce", utc=True)
    df["month"] = dt.dt.to_period("M").astype("string")

trend = (
    df.groupby("month", dropna=False)
    .agg(
        revenue=("amount","sum"),
        n=("order_id","size")
    )
    .reset_index()
    .sort_values("month")
)
print(trend)

     month  revenue  n
0  2025-12     45.5  4
1     <NA>    100.0  1


In [26]:
fig = px.line(trend, x="month", y="revenue", title="Revenue over time (monthly)")
fig.update_layout(title={"x":0.02})
fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Revenue")
save_fig(fig, FIGS / "revenue_trend_monthly.png")
fig.show()

# Q3
Amount distribution (winsorized)

In [27]:
if "amount_winsor" not in df.columns and "amount" in df.columns:
    lo, hi = df["amount"].quantile([0.01, 0.99])
    df["amount_winsor"] = df["amount"].clip(lo, hi)

In [28]:
fig = px.histogram(df, x="amount_winsor", nbins=30, title="Order amount distribution (winsorized)")
fig.update_layout(title={"x":0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of orders")
save_fig(fig, FIGS / "amount_hist_winsor.png")
fig.show()


# Q4
Data quality / Refund rate differences

In [29]:

df["is_refund"] = df["status"].eq("refund").astype(int)

a = df.loc[df["country"].eq("SA"), "is_refund"]
b = df.loc[df["country"].eq("AE"), "is_refund"]

print("n_SA:", len(a), "n_AE:", len(b))

def bootstrap_diff_means(a, b, *, n_boot=2000, seed=0):
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)
    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }

res = bootstrap_diff_means(a, b)
print(res)


n_SA: 4 n_AE: 1
{'diff_mean': -1.0, 'ci_low': -1.0, 'ci_high': -1.0}


In [60]:

data = pd.DataFrame({
    "Location": ["شارع محمد نور رحيمي، المهدية، غرب الرياض، الرياض", "شارع أبي عروة، المهدية، غرب الرياض، الرياض", "النهضة، شمال جدة، جدة"],
    "Type": ["Apartment", "villa", "Apartment"],
    "Number of Bedrooms": [3,4,5],
    "Number of Bathrooms": [3,5,4],
    "Price": [800000, 2150000, 1230000],
    "Area m^2": [126, 348, 186]
})

In [61]:
data

,Location,Type,Number of Bedrooms,Number of Bathrooms,Price,Area m^2
0,شارع محمد نور رحيمي، المهدية، غرب الرياض، الرياض,Apartment,3,3,800000,126
1,شارع أبي عروة، المهدية، غرب الرياض، الرياض,villa,4,5,2150000,348
2,النهضة، شمال جدة، جدة,Apartment,5,4,1230000,186


In [65]:
!pip install httpx

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.12.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)

   ---------------------------------------- 0/4 [h11]
   ---------------------------------------- 0/4 [h11]
   ---------------------------------------- 0/4 [h11]
   ---------------------------------------- 0/4 [h11]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ----------------------------- 1/4 [anyio]
   ---------- ---------------------

In [75]:
import httpx
result = httpx.get("https://api.thecatapi.com/v1/images/search?limit=10")
data = result.json()
df = pd.DataFrame(data)
df

,id,url,width,height
0,1ag,https://cdn2.thecatapi.com/images/1ag.gif,350,175
1,48f,https://cdn2.thecatapi.com/images/48f.gif,317,198
2,4bj,https://cdn2.thecatapi.com/images/4bj.gif,499,268
3,665,https://cdn2.thecatapi.com/images/665.png,1024,765
4,ar2,https://cdn2.thecatapi.com/images/ar2.jpg,2592,1944
5,bgq,https://cdn2.thecatapi.com/images/bgq.jpg,500,403
6,btf,https://cdn2.thecatapi.com/images/btf.jpg,640,428
7,ced,https://cdn2.thecatapi.com/images/ced.jpg,720,960
8,cg7,https://cdn2.thecatapi.com/images/cg7.jpg,500,342
9,rw09G0crt,https://cdn2.thecatapi.com/images/rw09G0crt.jpg,2400,1714


In [ ]:
import httpx
import pandas as pd

url = "THE_API_URL_FROM_NETWORK_TAB"
response = httpx.get(url)
data = response.json()

properties = []
for item in data["listings"]:  
    properties.append({
        "title": item.get("title"),
        "price": item.get("price"),
        "location": item.get("location"),
        "bedrooms": item.get("bedrooms"),
        "bathrooms": item.get("bathrooms"),
        "area_m2": item.get("area"),
        "url": item.get("url")
    })

df = pd.DataFrame(properties)
print(df.head())


In [135]:
import requests

cookies = {
    '_gcl_gs': '2.1.k1$i1766670491$u117709451',
    '_gcl_au': '1.1.2093967275.1766670493',
    'first_visit': 'true',
    'req-device-token': '6a7db2f0-e6d5-4478-aa04-35896362ee6d',
    '_gcl_aw': 'GCL.1766670494.CjwKCAiA3rPKBhBZEiwAhPNFQLcNGfhZg2GkF0w5OH-omOzjhIWeDTo2ilFFk1i718UEKQNz0NxfORoCV2UQAvD_BwE',
    '_scid': 'pSqDhlX636ixvYofqvydqIA_Twm3_Ixb',
    '_fbp': 'fb.1.1766670494560.729084752338938796',
    '_gid': 'GA1.2.322128027.1766670495',
    '_gac_UA-52229618-1': '1.1766670495.CjwKCAiA3rPKBhBZEiwAhPNFQLcNGfhZg2GkF0w5OH-omOzjhIWeDTo2ilFFk1i718UEKQNz0NxfORoCV2UQAvD_BwE',
    '_clck': '1kv3nhx%5E2%5Eg25%5E0%5E2185',
    '_tt_enable_cookie': '1',
    '_ttp': '01KDAW8V9Q4KRSD55PF6FBC59D_.tt.1',
    '_ScCbts': '%5B%5D',
    '_sctr': '1%7C1766610000000',
    '__cf_bm': 'isIbokbiI5nZGr2j.p3eCToi3HPPFo.B..jbpNI8SRA-1766671873-1.0.1.1-PL061zU7w8S3kBR3d3AF_v3lhIIUPQTdCvQOPPAw_ibOEqOI8SWYSH_PJjTK7Y87DHSFS9JRWYcyIXg71BoX9IrpjXth1QDVpU.OTojLTnY',
    '_ga_8DTTM22D6F': 'GS2.1.s1766670494$o1$g1$t1766671875$j59$l0$h0',
    '_ga': 'GA1.2.1062012610.1766670494',
    '_scid_r': 'sKqDhlX636ixvYofqvydqIA_Twm3_IxbZeSq6Q',
    'cf_clearance': 'oe9YRANTfkZafWxmfo.cr0fRzT0pDtdBaFYIL9pwZRg-1766671874-1.2.1.1-oclG.KEdJG4ousNALYLPJPVZ9cOAlG2z.pClv2nVW4nAdkdpIN5JHpAaTLb4b892cyVPNhVHu8bdU77jokioSpJNxxUjRieoqvWJ_VZ_d_q_s9mxBeTUGKteYQ7RHjGfWzjctfOf0B4Xd0_T61eQ9SWWrKUFp9eyXQbqkKuy1bMLi67L1K_KZUTMVdd7i3q4RIr3F7GWRNhrSoZlgNCxmuIL1c6uoH2Je0gHP1PG7D0',
    'ttcsid': '1766670495034::6EhsEyjUTSsWkJ2AqtJp.1.1766671876236.0',
    'ttcsid_CFI0VNBC77U110MKDP70': '1766670495033::l5Jznbiq4BQ8ASnRJVzr.1.1766671876236.1',
    '_clsk': '1rr32ud%5E1766671876258%5E5%5E1%5Ev.clarity.ms%2Fcollect',
}

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ar,en-US;q=0.9,en;q=0.8,sd;q=0.7',
    'cache-control': 'max-age=0',
    'priority': 'u=0, i',
    'sec-ch-ua': '"Google Chrome";v="143", "Chromium";v="143", "Not A(Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36',
    # 'cookie': '_gcl_gs=2.1.k1$i1766670491$u117709451; _gcl_au=1.1.2093967275.1766670493; first_visit=true; req-device-token=6a7db2f0-e6d5-4478-aa04-35896362ee6d; _gcl_aw=GCL.1766670494.CjwKCAiA3rPKBhBZEiwAhPNFQLcNGfhZg2GkF0w5OH-omOzjhIWeDTo2ilFFk1i718UEKQNz0NxfORoCV2UQAvD_BwE; _scid=pSqDhlX636ixvYofqvydqIA_Twm3_Ixb; _fbp=fb.1.1766670494560.729084752338938796; _gid=GA1.2.322128027.1766670495; _gac_UA-52229618-1=1.1766670495.CjwKCAiA3rPKBhBZEiwAhPNFQLcNGfhZg2GkF0w5OH-omOzjhIWeDTo2ilFFk1i718UEKQNz0NxfORoCV2UQAvD_BwE; _clck=1kv3nhx%5E2%5Eg25%5E0%5E2185; _tt_enable_cookie=1; _ttp=01KDAW8V9Q4KRSD55PF6FBC59D_.tt.1; _ScCbts=%5B%5D; _sctr=1%7C1766610000000; __cf_bm=isIbokbiI5nZGr2j.p3eCToi3HPPFo.B..jbpNI8SRA-1766671873-1.0.1.1-PL061zU7w8S3kBR3d3AF_v3lhIIUPQTdCvQOPPAw_ibOEqOI8SWYSH_PJjTK7Y87DHSFS9JRWYcyIXg71BoX9IrpjXth1QDVpU.OTojLTnY; _ga_8DTTM22D6F=GS2.1.s1766670494$o1$g1$t1766671875$j59$l0$h0; _ga=GA1.2.1062012610.1766670494; _scid_r=sKqDhlX636ixvYofqvydqIA_Twm3_IxbZeSq6Q; cf_clearance=oe9YRANTfkZafWxmfo.cr0fRzT0pDtdBaFYIL9pwZRg-1766671874-1.2.1.1-oclG.KEdJG4ousNALYLPJPVZ9cOAlG2z.pClv2nVW4nAdkdpIN5JHpAaTLb4b892cyVPNhVHu8bdU77jokioSpJNxxUjRieoqvWJ_VZ_d_q_s9mxBeTUGKteYQ7RHjGfWzjctfOf0B4Xd0_T61eQ9SWWrKUFp9eyXQbqkKuy1bMLi67L1K_KZUTMVdd7i3q4RIr3F7GWRNhrSoZlgNCxmuIL1c6uoH2Je0gHP1PG7D0; ttcsid=1766670495034::6EhsEyjUTSsWkJ2AqtJp.1.1766671876236.0; ttcsid_CFI0VNBC77U110MKDP70=1766670495033::l5Jznbiq4BQ8ASnRJVzr.1.1766671876236.1; _clsk=1rr32ud%5E1766671876258%5E5%5E1%5Ev.clarity.ms%2Fcollect',
}

response = requests.get('https://sa.aqar.fm/%D8%B9%D9%82%D8%A7%D8%B1%D8%A7%D8%AA', cookies=cookies, headers=headers)

In [136]:
#import httpx

#url = "https://sa.aqar.fm/%D8%B9%D9%82%D8%A7%D8%B1%D8%A7%D8%AA/1"
#response = httpx.get(url)

#html = response.text
html = response.text
assert "You have been blocked" not in html, "You have been blocked"

In [137]:
html

'<!DOCTYPE html><html lang="ar"><head><meta charSet="utf-8"/><link rel="preload" href="https://assets.aqar.fm/icons/ServiceList-blue.svg" as="image" fetchPriority="high"/><link rel="preload" href="https://assets.aqar.fm/icons/v1/aqar-logo-v2.svg" as="image" fetchPriority="high"/><link rel="preload" href="https://assets.aqar.fm/icons/red.svg" as="image" fetchPriority="high"/><link rel="preload" href="https://assets.aqar.fm/icons/dailyRent-colored.svg" as="image" fetchPriority="high"/><link rel="preload" href="https://assets.aqar.fm/icons/MapSearch-colored.svg" as="image" fetchPriority="high"/><link rel="preload" href="https://assets.aqar.fm/icons/Profile.svg" as="image" fetchPriority="high"/><title>عقارات | تطبيق عقار</title><link rel="dns-prefetch" href="https://assets.aqar.fm/"/><link rel="dns-prefetch" href="https://images.aqar.fm/"/><meta name="description" content="تصفح آلاف العقارات المعروضة للبيع والايجار في الرياض، جدة، الدمام، مكة، المدينة و جميع مناطق المملكة العربية السعودية 

In [138]:
!pip install beautifulsoup4


In [139]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")

In [140]:
soup

<!DOCTYPE html>
<html lang="ar"><head><meta charset="utf-8"/><link as="image" fetchpriority="high" href="https://assets.aqar.fm/icons/ServiceList-blue.svg" rel="preload"/><link as="image" fetchpriority="high" href="https://assets.aqar.fm/icons/v1/aqar-logo-v2.svg" rel="preload"/><link as="image" fetchpriority="high" href="https://assets.aqar.fm/icons/red.svg" rel="preload"/><link as="image" fetchpriority="high" href="https://assets.aqar.fm/icons/dailyRent-colored.svg" rel="preload"/><link as="image" fetchpriority="high" href="https://assets.aqar.fm/icons/MapSearch-colored.svg" rel="preload"/><link as="image" fetchpriority="high" href="https://assets.aqar.fm/icons/Profile.svg" rel="preload"/><title>عقارات | تطبيق عقار</title><link href="https://assets.aqar.fm/" rel="dns-prefetch"/><link href="https://images.aqar.fm/" rel="dns-prefetch"/><meta content="تصفح آلاف العقارات المعروضة للبيع والايجار في الرياض، جدة، الدمام، مكة، المدينة و جميع مناطق المملكة العربية السعودية بمساحات وأسعار مختل

In [141]:
soup.title

<title>عقارات | تطبيق عقار</title>

In [142]:
soup.title.name

'title'

In [143]:
soup.p

<p>مشروع ادوار سِـــدرا مشروع يتميز بموقعه الاستراتجي بحي النرجس جنوب طريق انس بن مالك وتصاميمه الفريدة.

دور ثاني بمساحة 169م

محتويات الدور:
-غرفة نوم ماستر 
-غرفة نوم 1
-غرفة نوم 2
-غرفة خادمة بدورة مياه
-صالة
-مطبخ
-سطح امامي 
-بلكونة عند الغرفة الماستر
-3 دورات مياه</p>

In [144]:
print(soup.get_text())

عقارات | تطبيق عقارالمشاريع العقارية الإيجار اليوميبحث بالخريطةإضافةالعقاراتالمشاريعالإيجار اليوميبحثالكلشقق للإيجارأراضي للبيعفلل للبيعدور للإيجارفلل للإيجارشقق للبيععمائر للبيعمحلات للإيجاربيت للبيعاستراحة للبيعبيت للإيجارمزرعة للبيعاستراحة للإيجارمكتب تجاري للإيجارأراضي للإيجارعمائر للإيجارمستودع للإيجارمخيم للإيجارغرف للإيجارمحلات للبيعدور للبيعشاليه للإيجارأخرىالرئيسيةعقاراتعقارات في الرياض (80716)عقارات في جدة (33773)عقارات في الدمام (7330)عقارات في الخبر (6091)عقارات في المدينة المنورة (4895)عقارات في مكة المكرمة (4004)المزيددور للبيع في شارع شهيد الدين ثم الوطن فهد إبراهيم علي أب, حي النرجس, مدينة الرياض, منطقة الرياض1,500,000 116 م²4 3 مشروع ادوار سِـــدرا مشروع يتميز بموقعه الاستراتجي بحي النرجس جنوب طريق انس بن مالك وتصاميمه الفريدة.

دور ثاني بمساحة 169م

محتويات الدور:
-غرفة نوم ماستر 
-غرفة نوم 1
-غرفة نوم 2
-غرفة خادمة بدورة مياه
-صالة
-مطبخ
-سطح امامي 
-بلكونة عند الغرفة الماستر
-3 دورات مياهالرياض-حي النرجسفيلا للبيع في شارع عذبة, حي حطين, مدينة الرياض, منطقة الرياض4,9

In [147]:
title_selector = '#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div > a > div > div._content__W4gas > div._titleRow__1AWv1 > h4'

In [ ]:
soup.select(title_selector)

[<h4>دور للبيع في شارع شهيد الدين ثم الوطن فهد إبراهيم علي أب, حي النرجس, مدينة الرياض, منطقة الرياض</h4>,
 <h4 class="_inline__nh3gU"><svg fill="none" height="14" viewbox="0 0 12 14" width="12" xmlns="http://www.w3.org/2000/svg"><path d="M12 5.94049C12 6.48383 11.8906 7.05433 11.6719 7.652C11.4531 8.24968 11.1344 8.8715 10.7156 9.51746C10.2969 10.1634 9.775 10.8305 9.15 11.5188C8.525 12.207 7.80625 12.9133 6.99375 13.6378C6.85625 13.7585 6.7 13.8491 6.525 13.9094C6.35 13.9698 6.175 14 6 14C5.825 14 5.65 13.9698 5.475 13.9094C5.3 13.8491 5.14375 13.7585 5.00625 13.6378C4.19375 12.9133 3.475 12.207 2.85 11.5188C2.225 10.8305 1.70312 10.1634 1.28437 9.51746C0.865625 8.8715 0.546875 8.24968 0.328125 7.652C0.109375 7.05433 0 6.48383 0 5.94049C0 4.12937 0.603125 2.6865 1.80938 1.6119C3.01562 0.537301 4.4125 0 6 0C7.5875 0 8.98438 0.537301 10.1906 1.6119C11.3969 2.6865 12 4.12937 12 5.94049Z" fill="#52B44A"></path><path d="M5.07194 6.80269L4.46763 6.17489C4.29496 5.99552 4.09353 5.90583 3.86

In [ ]:
title_elements =soup.select(title_selector) 

In [151]:
titles = []
for e in title_elements:
    titles.append(e.get_text())

In [152]:
titles

['دور للبيع في شارع شهيد الدين ثم الوطن فهد إبراهيم علي أب, حي النرجس, مدينة الرياض, منطقة الرياض',
 'فيلا للبيع في شارع عذبة, حي حطين, مدينة الرياض, منطقة الرياض',
 'فيلا للبيع في شارع خفاف بن ندبة, حي العليا, مدينة الرياض, منطقة الرياض',
 'فيلا للبيع في شارع 1809991, حي الروابي, مدينة الرياض, منطقة الرياض',
 'دور للإيجار في شارع شداد بن أوس, حي العليا, مدينة الرياض, منطقة الرياض',
 'شقة للبيع في شارع ربيعة القرشي, حي البوادي, مدينة جدة, منطقة مكة المكرمة',
 'دور للبيع في شارع الجارود السلمي, حي عكاظ, مدينة الرياض, منطقة الرياض',
 'فيلا للبيع في شارع ابن الغرس ، حي النيصية ، حائل ، منطقة حائل',
 'أرض للبيع في شارع ابي بكر ، حي الفيحاء ، جدة ، جدة',
 'عمارة للبيع في شارع عبدالسلام سحنون التنوخي ، حي الخالدية ، مكة ، منطقة مكة المكرمة',
 'عمارة للبيع في حي ، شارع عمر ابن يوسف المقرئ ، حي بطحاء قريش ، مكة ، منطقة مكة المكرمة',
 'عمارة للبيع في شارع أحمد بن عبد الواحد ، حي الكعكية ، مكة ، منطقة مكة المكرمة',
 'شقة للبيع في شارع منار الادب, حي السلامة, مدينة جدة, منطقة مكة المكرمة',
 'بيت 

In [213]:
s ="#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div:nth-child(1) > a > div > div._content__W4gas"

In [214]:
container= soup.select("#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div:nth-child(1) > a > div > div._content__W4gas")

In [216]:
container = container [0]

title = container.select_one("div._titleRow__1AWv1 h4").get_text(strip=True)
price = container.select_one("div._price__X51mi span").get_text(strip=True)
area = container.select_one("div._specs__nbsgm div._spec__SIJiK:nth-child(1)").get_text(strip=True)
bedrooms = container.select_one("div._specs__nbsgm div._spec__SIJiK:nth-child(2)").get_text(strip=True)
bathrooms = container.select_one("div._specs__nbsgm div._spec__SIJiK:nth-child(3)").get_text(strip=True)

property_info = {
    "title": title,
    "price": price,
    "area": area,
    "bedrooms": bedrooms,
    "bathrooms": bathrooms,
}

print(property_info)


{'title': 'دور للبيع في شارع شهيد الدين ثم الوطن فهد إبراهيم علي أب, حي النرجس, مدينة الرياض, منطقة الرياض', 'price': '1,500,000', 'area': '116 م²', 'bedrooms': '4', 'bathrooms': '3'}


In [236]:
def extract_info(container):
    container= soup.select(container[0])
    title = container.select_one("div._titleRow__1AWv1 h4").get_text(strip=True)
    price = container.select_one("div._price__X51mi span").get_text(strip=True)
    area = container.select_one("div._specs__nbsgm div._spec__SIJiK:nth-child(1)").get_text(strip=True)
    bedrooms = container.select_one("div._specs__nbsgm div._spec__SIJiK:nth-child(2)").get_text(strip=True)
    bathrooms = container.select_one("div._specs__nbsgm div._spec__SIJiK:nth-child(3)").get_text(strip=True)
    property_info = {
        "title": title,
        "price": price,
        "area": area,
        "bedrooms": bedrooms,
        "bathrooms": bathrooms,
        }


In [237]:
s1 =extract_info("#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div:nth-child(1) > a > div > div._content__W4gas")
s2 = extract_info("#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div:nth-child(2) > a > div > div._content__W4gas")

SelectorSyntaxError: Malformed id selector at position 0
  line 1:
#
^

In [231]:
s1

{'title': 'دور للبيع في شارع شهيد الدين ثم الوطن فهد إبراهيم علي أب, حي النرجس, مدينة الرياض, منطقة الرياض',
 'price': '1,500,000',
 'area': '116 م²',
 'bedrooms': '4',
 'bathrooms': '3'}

In [225]:
s2

{'title': 'فيلا للبيع في شارع عذبة, حي حطين, مدينة الرياض, منطقة الرياض',
 'price': '4,900,000',
 'area': '300 م²',
 'bedrooms': '6',
 'bathrooms': '3'}

In [226]:
S= "#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A"

In [233]:
s = extract_info(S)

TypeError: unhashable type: 'ResultSet'

In [238]:
s = "#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div:nth-child(1) > a > div > div._content__W4gas"

In [239]:
s

'#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div:nth-child(1) > a > div > div._content__W4gas'

In [240]:
s2= "#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div:nth-child(1) > a > div > div._content__W4gas > div._titleRow__1AWv1 > h4"

In [241]:
s2

'#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div:nth-child(1) > a > div > div._content__W4gas > div._titleRow__1AWv1 > h4'

In [243]:
s3 = "#__next > main > div > div._root__Szbd6 > div > div:nth-child(2) > div._container__Lu67A > div._list__Ka30R > div:nth-child(1) > a > div > div._content__W4gas > div._price__X51mi > span"